In [1]:
import os
import pandas as pd 
import numpy as np 
import scipy as scp
import sklearn

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn import metrics 
from sklearn.metrics import confusion_matrix

import statsmodels.api as sm
import matplotlib.pyplot as plt

#from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import GridSearchCV

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]=""

# 📂DATA : 2 Classes

In [77]:
fold = 6

In [78]:
path = "/home/kannika/code/Rheology2023/Rheology_Blood/DataBlood_Viscosity_TrainML_6Fold_split2class.csv"
data_feature = pd.read_csv(path)
print(data_feature.shape)
print("-"*100)
print(f"All Fold : {set(data_feature.fold)}")
## Split Train data Set
feature_train = data_feature[data_feature["fold"]!=fold].reset_index(drop=True)
print(f"Train Set : Fold ==> {set(feature_train.fold)}")
print("Train = ", feature_train.shape)
## Split Valid data Set
feature_test = data_feature[data_feature["fold"]==fold].reset_index(drop=True)
print(f"Valiadtion Set : Fold ==> {set(feature_test.fold)}")
print("Validation = ", feature_test.shape)
## Print DataFrame
feature_train.head()

(51, 9)
----------------------------------------------------------------------------------------------------
All Fold : {1, 2, 3, 4, 5, 6}
Train Set : Fold ==> {1, 2, 3, 4, 5}
Train =  (43, 9)
Valiadtion Set : Fold ==> {6}
Validation =  (8, 9)


,Unnamed: 0,Code,classes,subclass,fold,classes_binary,MCV,MCH,Hb
0,0,HN35,HN,No_Splenectomy,1,1.0,74.3,23.4,7.1
1,1,HN43,HN,No_Splenectomy,1,1.0,57.4,17.7,4.4
2,2,HN38,HN,No_Splenectomy,1,1.0,72.9,22.6,8.0
3,3,NBL14,NBL,Normal,1,0.0,86.8,28.1,13.2
4,4,NBL15,NBL,Normal,1,0.0,88.2,27.9,13.9


In [79]:
X_train = feature_train[['MCV','MCH','Hb']]
y_train = feature_train["classes"]
print(X_train.shape)
print(y_train.shape)
print(y_train[0])

(43, 3)
(43,)
HN


In [80]:
y_train = feature_train["classes"]
print(len(list(set(y_train))))
print(set(y_train))

2
{'HN', 'NBL'}


## 🩸 Parameter:  Logistic Regression

In [81]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report,confusion_matrix
import warnings
warnings.filterwarnings('ignore')

In [82]:
LR = LogisticRegression(random_state = 1)

LRparam_grid = {
    'C' : [0.001, 0.01, 0.1, 1],
    'penalty': ['l1', 'l2'],
    'max_iter': list(range(100,500,100)),
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}

LR_search = GridSearchCV(LR, param_grid=LRparam_grid, refit = True, verbose = 1, cv=10, n_jobs = -1)
# fitting the model for grid search 
best_LR = LR_search.fit(X_train, y_train)

Fitting 10 folds for each of 160 candidates, totalling 1600 fits


In [83]:
# view the results as a pandas DataFrame
best_LRdf = pd.DataFrame(best_LR.cv_results_)[['rank_test_score', 'mean_test_score', 'std_test_score', 'params']]
best_LRdf = best_LRdf.sort_values(by='rank_test_score', ascending=True)
best_LRdf = best_LRdf.reset_index(drop=True)
best_LRdf.head()

,rank_test_score,mean_test_score,std_test_score,params
0,1,1.0,0.0,"{'C': 1, 'max_iter': 300, 'penalty': 'l2', 'so..."
1,1,1.0,0.0,"{'C': 1, 'max_iter': 400, 'penalty': 'l2', 'so..."
2,1,1.0,0.0,"{'C': 1, 'max_iter': 400, 'penalty': 'l2', 'so..."
3,1,1.0,0.0,"{'C': 1, 'max_iter': 200, 'penalty': 'l2', 'so..."
4,1,1.0,0.0,"{'C': 1, 'max_iter': 200, 'penalty': 'l2', 'so..."


In [84]:
# examine the first result
print("**examine the first result","\n")

print(best_LR.cv_results_['params'][0])
print(best_LR.cv_results_['mean_test_score'][0])

# print the array of mean scores only
print("\n","**print the array of mean scores only","\n")

grid_mean_scores = best_LR.cv_results_['mean_test_score']
print(grid_mean_scores)

# examine the best model
print("\n","**examine the best model","\n")

print(best_LR.best_score_)
print(best_LR.best_params_)
print(best_LR.best_estimator_)

**examine the first result 

{'C': 0.001, 'max_iter': 100, 'penalty': 'l1', 'solver': 'newton-cg'}
nan

 **print the array of mean scores only 

[  nan   nan 0.705   nan 0.705 0.82  0.82  0.705 0.705 0.705   nan   nan
 0.705   nan 0.705 0.82  0.82  0.705 0.705 0.705   nan   nan 0.705   nan
 0.705 0.82  0.82  0.705 0.705 0.705   nan   nan 0.705   nan 0.705 0.82
 0.82  0.705 0.705 0.705   nan   nan 0.705   nan 0.705 0.905 0.905 0.775
 0.775 0.775   nan   nan 0.705   nan 0.705 0.905 0.905 0.775 0.775 0.775
   nan   nan 0.705   nan 0.705 0.905 0.905 0.775 0.775 0.775   nan   nan
 0.705   nan 0.705 0.905 0.905 0.775 0.775 0.775   nan   nan 0.93    nan
 0.795 0.95  0.95  0.905 0.88  0.835   nan   nan 0.93    nan 0.835 0.95
 0.95  0.905 0.93  0.88    nan   nan 0.93    nan 0.855 0.95  0.95  0.905
 0.93  0.93    nan   nan 0.93    nan 0.88  0.95  0.95  0.905 0.93  0.93
   nan   nan 0.93    nan 0.855 1.    1.    0.93  0.93  0.855   nan   nan
 0.93    nan 0.93  1.    1.    0.93  0.905 0.93    nan 

In [85]:
#Print the tured parameters and score
print("Tuned Decision Tree Parameters: {}".format(best_LR.best_params_))
print("Best score is {}".format(best_LR.best_score_))

Tuned Decision Tree Parameters: {'C': 1, 'max_iter': 100, 'penalty': 'l2', 'solver': 'newton-cg'}
Best score is 1.0


In [87]:
best_params_ =  best_LR.best_params_
C_ = best_params_['C'] 
max_iter_ = best_params_['max_iter'] 
penalty_ = best_params_['penalty']   
solver_ = best_params_['solver']

> ## 🚀 Fit Model and setting parameters

In [88]:
## **- train Fit Model XGBOOST
Logistic = LogisticRegression(random_state = 1, C=C_, max_iter=max_iter_, penalty=penalty_, solver=solver_)
LogisticModel = Logistic.fit(X_train, y_train) # train XGBOOST model
LogisticModel

LogisticRegression(C=1, random_state=1, solver='newton-cg')

In [89]:
### Testing Check Parameter
print(LogisticModel.get_params())

{'C': 1, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': 1, 'solver': 'newton-cg', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


> ## #️⃣ Save model

In [90]:
numclass = 2

In [91]:
#Save Model
import os
import pickle
import imageio

save_pathimg = f'/media/tohn/HDD/rheology2023/ML_Model/Blood_Viscosity/_{numclass}Classes/fold{fold}'
##**Mkdir Directory 
os.makedirs(save_pathimg, exist_ok=True)     
filename = f"Logis_Model_Blood{numclass}Class_fold{fold}.pkl"
Model2Save =  f"{save_pathimg}/{filename}"
print(f"[INFO]: Done!! Save Model as : {Model2Save}")

with open(Model2Save, 'wb') as file:
    pickle.dump(LogisticModel, file)

[INFO]: Done!! Save Model as : /media/tohn/HDD/rheology2023/ML_Model/Blood_Viscosity/_2Classes/fold6/Logis_Model_Blood2Class_fold6.pkl
